# Import Library

In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from tqdm import tqdm

import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont
from IPython.display import display

# This is needed since the notebook is stored in the object_detection folder.
# sys.path.append("..")

# Import utilities
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

2022-10-24 07:59:27.893745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 07:59:27.893788: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Define Prediction Function

In [2]:
# Digit Prediction functions methods definition

def convert_range(prediction_score, OldMin, OldMax):
    NewMax = 1
    NewMin = 0
    
    OldRange = (OldMax - OldMin)  
    NewRange = (NewMax - NewMin)  
    NewValue = (((prediction_score - OldMin) * NewRange) / OldRange) + NewMin
    return NewValue

def get_prediction_s(image_dir, model_name):
    test_img = tf.keras.utils.load_img(image_dir,
                                   grayscale = False,
                                   color_mode = 'rgb',
                                   target_size = (160,160,3),
                                   interpolation = 'nearest')
    img_array = tf.keras.utils.img_to_array(test_img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model_name.predict(img_array)
    score = predictions[0]
    
    class_names = [int(i) for i in range(10)]
    pred_label = class_names[np.argmax(score)]
    
    conf_score = np.max(score)
    model_score = np.max(score)
    
    return pred_label, conf_score, model_score,score

# Load Models

### Load Localization Model & Label Map

In [3]:
# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
# Grab path to current working directory

CWD_PATH = os.getcwd()

NUM_CLASSES = 1

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,'ml_models',MODEL_NAME,'221005_frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.compat.v1.Session(graph=detection_graph)

2022-10-24 08:02:10.495908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.7/site-packages/cv2/../../lib64:
2022-10-24 08:02:10.495955: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-24 08:02:10.495981: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (addin-gurihmas-dev): /proc/driver/nvidia/version does not exist
2022-10-24 08:02:10.496476: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

### Load Prediction Model

In [5]:
# Load digit prediction trained model
model = tf.keras.models.load_model(os.path.join(CWD_PATH,'ml_models','DR_MobileNetV2_v3.h5'))

# Show the model architecture
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810 

## List Sample Images

In [6]:
data_to_crop = os.listdir(os.path.join(CWD_PATH,'image_samples/image_asli_added'))
data_to_crop = pd.DataFrame(data_to_crop,columns=['filename'])
data_to_crop.to_csv('/home/jupyter/gurih_mas/OCR/results/221018_filename_result.csv',index=False)
print(data_to_crop.shape)
data_to_crop.head()

(300, 1)


,filename
0,0068ef20-1d1b-4961-af65-7d9d0a0897cc.jpg
1,006404a7-81d4-4d8c-9dbe-ce3c8f4df4e8.jpg
2,00ffe211-568a-4345-ba25-0829863d0f7d.jpg
3,006002a2-1345-4e49-97c5-f3b08f01852e.jpg
4,003e5596-fa43-48ae-b5ce-5fd389d25128.jpg


## Localize Digit Image

In [7]:
CWD_PATH = os.getcwd()
image_path = os.path.join(CWD_PATH,'image_samples/image_asli_added')
image_cropped_path = os.path.join(CWD_PATH,'image_samples/cropped_image')
image_localized_path = os.path.join(CWD_PATH,'image_samples/localized_image')
digit_dataset_path = os.path.join(CWD_PATH,'image_samples/digit_dataset')

filename_list = []
filename_crop_list = [] 
ymin_list = []
ymax_list = []
xmin_list = []
xmax_list = []
width_list = []
height_list = []

# IMAGE_NAME = '10b189c6-8a5a-400d-9734-1800be3466d4.jpg'

for i in tqdm(range(data_to_crop.shape[0])):
    IMAGE_NAME = data_to_crop.filename[i]
    image_name_raw = IMAGE_NAME.split('.')[-2]
    
    # PATH_TO_IMAGE = os.path.join(CWD_PATH,'sampel_gambar','sample_data_real_test',IMAGE_NAME)
    PATH_TO_IMAGE = os.path.join(image_path,IMAGE_NAME)
    
    image = cv2.imread(PATH_TO_IMAGE)
    raw_image = cv2.imread(PATH_TO_IMAGE)
    height, width, channels = image.shape
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_expanded = np.expand_dims(image_rgb, axis=0)
    
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})

    final_image, bbox = vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=3,
        min_score_thresh=0.95)

    # display(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))
    im = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    im.save(image_localized_path+'/{}.jpg'.format(image_name_raw,i))

    bbox.sort(reverse=False)

    for i in range(len(bbox)):
        xmin = bbox[i][0]
        ymin = bbox[i][1]
        xmax = bbox[i][2]
        ymax = bbox[i][3]

        filename_list.append(IMAGE_NAME)
        ymin_list.append(ymin)
        xmin_list.append(xmin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)
        width_list.append(width)
        height_list.append(height)

        cropped_img = raw_image[ymin:ymax,xmin:xmax]
        rgb_cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)

        # display(Image.fromarray(rgb_cropped_img))
        im_crop = Image.fromarray(rgb_cropped_img)
        im_crop.save(image_cropped_path+'/{}_{}.jpg'.format(image_name_raw,i))
        filename_crop_list.append('{}_{}.jpg'.format(image_name_raw,i))

100%|██████████| 300/300 [05:35<00:00,  1.12s/it]


In [8]:
summary_df = pd.DataFrame([])
summary_df['filename_ktp'] = filename_list
summary_df['filename_digit'] = filename_crop_list
print('Total cropped images: {}'.format(summary_df.shape))

Total cropped images: (3825, 2)


In [9]:
summary_df.to_csv(os.path.join(CWD_PATH,'image_samples/221018_added_cropped_result.csv'),index=False)

## Predict Digit Image

In [10]:
pred_label=[]
scoring=[]

for i in tqdm(range(len(filename_crop_list))):
    image_filename = filename_crop_list[i]
    image_full_path_dir = '{}/{}'.format(image_cropped_path, image_filename)

    try:
        pred_label_s, conf_score_s, model_score_s,score = get_prediction_s(image_full_path_dir, model)
        pred_label.append(pred_label_s)
        scoring.append(conf_score_s)
    except:
        pass

100%|██████████| 3825/3825 [03:24<00:00, 18.74it/s]


In [11]:
summary_df['pred_score'] = scoring
summary_df['pred_label'] = pred_label
print('Predicted images: {}'.format(summary_df.shape))
summary_df.head()

Predicted images: (3825, 4)


,filename_ktp,filename_digit,pred_score,pred_label
0,0068ef20-1d1b-4961-af65-7d9d0a0897cc.jpg,0068ef20-1d1b-4961-af65-7d9d0a0897cc_0.jpg,0.999660,3
1,0068ef20-1d1b-4961-af65-7d9d0a0897cc.jpg,0068ef20-1d1b-4961-af65-7d9d0a0897cc_1.jpg,0.999271,5
2,0068ef20-1d1b-4961-af65-7d9d0a0897cc.jpg,0068ef20-1d1b-4961-af65-7d9d0a0897cc_2.jpg,1.000000,0
3,0068ef20-1d1b-4961-af65-7d9d0a0897cc.jpg,0068ef20-1d1b-4961-af65-7d9d0a0897cc_3.jpg,1.000000,7
4,0068ef20-1d1b-4961-af65-7d9d0a0897cc.jpg,0068ef20-1d1b-4961-af65-7d9d0a0897cc_4.jpg,0.999856,1


In [12]:
summary_df.to_csv(os.path.join(CWD_PATH,'image_samples/221018_added_prediction_result.csv'),index=False)

## Copy predicted images to each labels

In [13]:
class_names = [str(i) for i in range(10)]

for c in class_names:
    if os.path.isdir(os.path.join(digit_dataset_path,c)) == False:
        os.makedirs(os.path.join(digit_dataset_path,c))
    else:
        pass

In [14]:
#Copy images based on labels

for i in tqdm(range(summary_df.shape[0])):
    img = summary_df['filename_digit'][i]
    label = str(summary_df['pred_label'][i])
    source_final_path = os.path.join(image_cropped_path,img)
    dest_final_path = os.path.join(digit_dataset_path,label,img)
    if os.path.isfile(dest_final_path) == False:
        shutil.copy(source_final_path,dest_final_path)

100%|██████████| 3825/3825 [00:00<00:00, 55746.44it/s]


## Summarize digit dataset

In [15]:
summary_df.pred_label.value_counts().sort_index()

0    1646
1     471
2     258
3     435
4      98
5     238
6     147
7     256
8     123
9     153
Name: pred_label, dtype: int64

In [16]:
df_digit = summary_df.filter(['filename_digit','pred_label'], axis=1).sort_values('pred_label').reset_index(drop=True)
df_digit

,filename_digit,pred_label
0,0068f939-87a8-43e6-b5f1-202173afe9cc_13.jpg,0
1,002409f6-cfc7-494c-ad42-ba82ec1fc0c7_13.jpg,0
2,002409f6-cfc7-494c-ad42-ba82ec1fc0c7_14.jpg,0
3,01717470-8425-431c-bf42-f048957329be_3.jpg,0
4,01717470-8425-431c-bf42-f048957329be_4.jpg,0
...,...,...
3820,01523787-3ec3-4160-bfcc-6faeda7b2ff3_10.jpg,9
3821,00a985d0-6125-4b14-88a7-5c6a895d5eb9_11.jpg,9
3822,0097eec9-3806-42e0-9308-44ec0703e7dd_5.jpg,9
3823,0123f301-2ec5-45e5-bf63-ffb79c16e252_9.jpg,9


In [17]:
class_names = list(df_digit.pred_label.unique()) #Create a Python list of Unique labels in data frame labels

In [27]:
#Split dataframe into multiple dataframe based on unique column values
dfdigit0 = df_digit.loc[df_digit['pred_label'] == 0]
dfdigit0 = dfdigit0[0:146]
dfdigit3 = df_digit.loc[df_digit['pred_label'] == 3]
dfdigit3 = dfdigit3[0:114]
dfdigit9 = df_digit.loc[df_digit['pred_label'] == 9]
dfdigit9 = dfdigit9[0:150]

In [29]:
#Splitting images for each PIC
pic = ['Addin', 'Vanda']
list_pic0 = []
list_pic3 = []
list_pic9 = []

for member in pic :
    for i in range(int(146/2)):
        list_pic0.append(member)
dfdigit0['PIC'] = list_pic0

for member in pic :
    for i in range(int(114/2)):
        list_pic3.append(member)
dfdigit3['PIC'] = list_pic3

for member in pic :
    for i in range(int(150/2)):
        list_pic9.append(member)
dfdigit9['PIC'] = list_pic9

In [30]:
filename_dataset = pd.concat([dfdigit0, dfdigit3, dfdigit9], axis=0).reset_index(drop=True)
filename_dataset.to_csv('221018_Added_Image_Samples.csv', index=False)
filename_dataset.pred_label.value_counts().sort_index()

0    146
3    114
9    150
Name: pred_label, dtype: int64

In [31]:
filename_dataset.PIC.value_counts().sort_index()

Addin    205
Vanda    205
Name: PIC, dtype: int64

In [32]:
filename_dataset

,filename_digit,pred_label,PIC
0,0068f939-87a8-43e6-b5f1-202173afe9cc_13.jpg,0,Addin
1,002409f6-cfc7-494c-ad42-ba82ec1fc0c7_13.jpg,0,Addin
2,002409f6-cfc7-494c-ad42-ba82ec1fc0c7_14.jpg,0,Addin
3,01717470-8425-431c-bf42-f048957329be_3.jpg,0,Addin
4,01717470-8425-431c-bf42-f048957329be_4.jpg,0,Addin
...,...,...,...
405,0123f301-2ec5-45e5-bf63-ffb79c16e252_11.jpg,9,Vanda
406,00a985d0-6125-4b14-88a7-5c6a895d5eb9_10.jpg,9,Vanda
407,00e8f929-b97c-4dd7-b34e-dd63a497f642_2.jpg,9,Vanda
408,01523787-3ec3-4160-bfcc-6faeda7b2ff3_10.jpg,9,Vanda


### Split images to each PIC

In [33]:
# Create directory for each PIC
class_names = [str(i) for i in range(10)]
pic = ['Addin', 'Vanda']

for p in pic:
    for c in class_names:
        if os.path.isdir(os.path.join('/home/jupyter/gurih_mas/OCR/image_samples',p,c)) == False:
            os.makedirs(os.path.join('/home/jupyter/gurih_mas/OCR/image_samples',p,c))
        else:
            pass

In [35]:
# Copy images based on PIC

dest_path = '/home/jupyter/gurih_mas/OCR/image_samples'

for i in tqdm(range(filename_dataset.shape[0])):
    img = filename_dataset['filename_digit'][i]
    pic = filename_dataset['PIC'][i]
    label = str(filename_dataset['pred_label'][i])
    source_final_path = os.path.join(image_cropped_path,img)
    dest_final_path = os.path.join(dest_path,pic,label,img)
    if os.path.isfile(dest_final_path) == False:
        shutil.copy(source_final_path,dest_final_path)

100%|██████████| 410/410 [00:00<00:00, 7244.78it/s]


In [91]:
import os
import zipfile

pic = ['Mas Kemas', 'Addin', 'Krisna', 'Rio', 'Vanda']

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

for p in pic :
    with zipfile.ZipFile('/home/jupyter/OCR/addin/digit_localization_prediction_model/{}.zip'.format(p), 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipdir('/home/jupyter/OCR/addin/digit_localization_prediction_model/sampel_gambar/221018_dataset/{}'.format(p), zipf)

In [36]:
def make_archive(source, destination):
    base = os.path.basename(destination)
    name = base.split('.')[0]
    format = base.split('.')[1]
    archive_from = os.path.dirname(source)
    archive_to = os.path.basename(source.strip(os.sep))
    print(source, destination, archive_from, archive_to)
    shutil.make_archive(name, format, archive_from, archive_to)
    shutil.move('%s.%s'%(name,format), destination)

In [37]:
pic = ['Addin', 'Vanda']

for p in pic :
    make_archive(os.path.join('/home/jupyter/gurih_mas/OCR/image_samples/pic_split/221018_added_dataset',p), '/home/jupyter/gurih_mas/OCR/image_samples/pic_split/221018_added_dataset/{}.zip'.format(p))

/home/jupyter/gurih_mas/OCR/image_samples/Addin /home/jupyter/gurih_mas/OCR/Addin.zip /home/jupyter/gurih_mas/OCR/image_samples Addin
/home/jupyter/gurih_mas/OCR/image_samples/Vanda /home/jupyter/gurih_mas/OCR/Vanda.zip /home/jupyter/gurih_mas/OCR/image_samples Vanda
